# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2021spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1aQoWogAQo_xVJvMQMrGaYiWzuyfO0QyLLAhiMwFyS2w)　Kaggle: [Link](https://www.kaggle.com/c/ml2021-spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

In [1]:
import torch

torch.cuda.get_device_name()

'Tesla V100-SXM2-16GB'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Download Dataset

In [3]:
# Download link 1
!gdown --id '1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1pOu3FdPdvzielUZyggeD7KDnVy9iW1uC' --output hw7_data.zip

!unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
#!nvidia-smi

Downloading...
From: https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1
To: /content/hw7_data.zip
7.71MB [00:00, 21.2MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          


In [13]:
!nvidia-smi

Sun May 16 08:42:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    54W / 300W |  16143MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
!ps -aux|grep python

root          93  0.0  0.0 128668 16556 ?        Sl   02:30   0:04 /usr/bin/python3 /usr/local/lib/python3.7/dist-packages/debugpy/adapter --for-server 46587 --host 127.0.0.1 --port 15157 --server-access-token ee8a1401e7fabfc136084b2ad7cfd3fc66580c38f05518c5b62c64481f7107a7
root         256  0.0  0.0  18384  3096 ?        S    02:30   0:00 bash -c tail -n +0 -F "/root/.config/Google/DriveFS/Logs/drive_fs.txt" | python3 /opt/google/drive/drive-filter.py > "/root/.config/Google/DriveFS/Logs/timeouts.txt" 
root         258  0.0  0.0  31744  9552 ?        S    02:30   0:00 python3 /opt/google/drive/drive-filter.py
root        1156  0.0  0.0      0     0 ?        Z    03:42   0:01 [python3] <defunct>
root        1315  0.0  0.0      0     0 ?        Z    03:49   0:02 [python3] <defunct>
root        1479  0.0  0.0      0     0 ?        Zs   04:01   0:01 [python3] <defunct>
root        1499  0.0  0.0      0     0 ?        Z    04:01   0:00 [python3] <defunct>
root        1525  1.1  0.2 197508 

In [ ]:
!kill -9 5069

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [3]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

## Import Packages

In [1]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

In [2]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

## Load Model and Tokenizer




 

In [3]:

from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForPreTraining

# model = BertForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")


tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-roberta-wwm-ext-large")
model = BertForQuestionAnswering.from_pretrained("hfl/chinese-roberta-wwm-ext-large").to(device)

#model = BertForQuestionAnswering.from_pretrained("hfl/chinese-macbert-large").to(device)
#tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-macbert-large")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext-large were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl

## Read Data

- Training set: 26935 QA pairs
- Dev set: 3523  QA pairs
- Test set: 3492  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [4]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [5]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

In [6]:
# 句號的encode

dotend = tokenizer.encode('。', add_special_tokens=False)[0]
dotend

511

In [7]:
comma = tokenizer.encode('，', add_special_tokens=False)[0]
comma

8024

## Dataset and Dataloader

In [33]:
test_list = tokenizer(train_paragraphs[5870], add_special_tokens=False).input_ids
train_paragraphs[5870]

In [35]:
for i in train_questions:
  if i['paragraph_id'] == 5870:
    print(i)

{'id': 2566, 'paragraph_id': 5870, 'question_text': 'PlayStation發售後3年內穩居遊戲市場第一的位置是因為哪一款遊戲?', 'answer_text': 'Final Fantasy系列', 'answer_start': 476, 'answer_end': 490}
{'id': 7755, 'paragraph_id': 5870, 'question_text': '「索尼互動娛樂」商標上的方形是甚麼顏色?', 'answer_text': '黃色', 'answer_start': 747, 'answer_end': 748}
{'id': 9783, 'paragraph_id': 5870, 'question_text': '索尼互動娛樂在哪一年成立?', 'answer_text': '1993年', 'answer_start': 41, 'answer_end': 45}
{'id': 25385, 'paragraph_id': 5870, 'question_text': '任天堂與索尼的合作在哪一年正式決裂?', 'answer_text': '1992年', 'answer_start': 164, 'answer_end': 168}


In [43]:
len(train_paragraphs[5870])

755

In [45]:
size = len(test_list)
idx_list = [idx + 1 for idx, val in
            enumerate(test_list) if val == dotend] 
print(idx_list)
res = [test_list[i: j] for i, j in
        zip([0] + idx_list, idx_list + 
        ([size] if idx_list[-1] != size else []))]
  
# print result

total_window = len(res)
counter = 50

for i, window in enumerate(res):
  

  counter += len(window)
  print('w', counter)
  if train_questions[2566]['answer_start'] < counter:
    print(len(window))
    print('a',train_questions[2566]['answer_start'])
    start = max(0, counter - len(window) - 50)
    end = (counter)
    print(test_list[start:end])
    break
  else:
    start = counter - 100
    end = counter + 100
    

print('c',end)
    

[81, 103, 148, 195, 222, 247, 350, 445, 462, 495, 524, 543, 602]
w 131
w 153
w 198
w 245
w 272
w 297
w 400
w 495
95
a 476
[8447, 2399, 8024, 11515, 8154, 809, 4534, 3229, 3297, 1044, 4999, 4638, 8219, 2512, 1008, 2825, 6123, 4158, 3636, 1690, 4638, 7442, 6213, 6879, 3556, 1690, 519, 11026, 520, 1217, 1057, 4801, 7768, 2356, 1842, 8024, 5645, 9342, 8676, 4638, 9342, 8676, 8536, 12760, 1469, 818, 1921, 1828, 4638, 10560, 8511, 11652, 8167, 8308, 2245, 7274, 749, 4080, 4164, 4638, 2356, 1842, 5000, 4261, 8039, 738, 1728, 4158, 10591, 12436, 5143, 1154, 4638, 2512, 7513, 8024, 11026, 4634, 1545, 2527, 124, 2399, 1058, 4952, 2233, 6879, 2783, 2356, 1842, 5018, 671, 4638, 855, 5390, 511, 8258, 2399, 8024, 11515, 8154, 4634, 1545, 3108, 2380, 6879, 2783, 712, 3582, 11026, 11311, 9609, 511, 8213, 2399, 8111, 3299, 8024, 11515, 8154, 4634, 1545, 11026, 124, 8024, 1728, 4158, 4634, 6121, 6733, 2714, 510, 5965, 1045, 1045, 4817, 1019, 3419, 3203, 6523, 5445, 671, 2428, 3760, 5862, 511]
c 495


In [12]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        #self.max_paragraph_len = 150
        #self.max_paragraph_len = 450 #0.8
        self.max_paragraph_len = 300 #0.82 0.807

        ##### TODO: Change value of doc_stride #####
        #self.doc_stride = 150
        #self.doc_stride = 10 #0.8
        self.doc_stride = 25 #0.82 0.807
        #self.doc_stride = 300


        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx] # 一次取一個問題
        tokenized_question = self.tokenized_questions[idx] #一次取一個問題的token
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]] # 找出對印的文章[文章id]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"]) # 看答案的開頭在段落的哪個位置
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"]) # 看答案的結尾在段落的哪個位置

            # A single window is obtained by slicing the portion of paragraph containing the answer
            #print(tokenized_paragraph.ids)

            #mid = (answer_start_token + answer_end_token) // 2
            #paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))


           
            # rand = np.random.choice([100, 160, 190, 200, 210 ,220, 250, 300], 1)[0]

            # paragraph_start = max(0,answer_start_token - rand)
            # paragraph_end = paragraph_start + self.max_paragraph_len
            try:
              size = len(tokenized_paragraph.ids)
              idx_list = [idx + 1 for idx, val in enumerate(tokenized_paragraph.ids) if val == dotend] 

              res = [tokenized_paragraph.ids[i: j] for i, j in zip([0] + idx_list, idx_list + ([size] if idx_list[-1] != size else []))]

              # print result
              counter = 50

              for i, window in enumerate(res):
                counter += len(window)
                if question["answer_start"] < counter:
                  paragraph_start = max(0, counter - len(window) - 50)
                  paragraph_end = counter
                  if (paragraph_end - paragraph_start) > 300:
                    rand = np.random.choice([100, 160, 190, 200, 210 ,220, 250, 300], 1)[0]
                    paragraph_start = max(0,answer_start_token - rand)
                    paragraph_end = paragraph_start + self.max_paragraph_len 
                  break
                else:
                  paragraph_start = counter - 120
                  paragraph_end = counter + 120
              

            except:
              size = len(tokenized_paragraph.ids)
              idx_list = [idx + 1 for idx, val in enumerate(tokenized_paragraph.ids) if val == comma] 

              res = [tokenized_paragraph.ids[i: j] for i, j in zip([0] + idx_list, idx_list + ([size] if idx_list[-1] != size else []))]

              # print result
              total_window = len(res)
              counter = 100

              for i, window in enumerate(res):
                counter += len(window)
                if question["answer_start"] < counter:

                  paragraph_start = max(0, counter - len(window) - 100)
                  paragraph_end = counter
                  if (paragraph_end - paragraph_start) > 400:
                    rand = np.random.choice([100, 160, 190, 200, 210 ,220, 250, 300], 1)[0]
                    paragraph_start = max(0,answer_start_token - rand)
                    paragraph_end = paragraph_start + self.max_paragraph_len 
                  break
                else:
                  paragraph_start = counter - 150
                  paragraph_end = counter + 150
            


            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 


            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride): # 文章總長度
 
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] # ids取出問題
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102] # ids取出文章
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 4

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [10]:
def evaluate(data, output):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]


    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)

        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            if start_index < end_index:
              
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
              answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
            else:
              answer = ''

    # Remove spaces in answer (e.g. "大 金" --> "大金")

    return answer.replace(' ','')


## Training

In [13]:
num_epoch = 2
validation = True
logging_step = 500
learning_rate = 2e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 3000, gamma=0.6, last_epoch=-1)
if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 
#accumulation_steps = 4
model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    learning_rate = 2e-5
    train_loss = train_acc = 0
    
    for data in tqdm(train_loader):	

        # Load all data into GPU
        data = [i.to(device) for i in data]

        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  

        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss.item()
        #train_loss = output.loss.item() / accumulation_steps
        if fp16_training:
            accelerator.backward(output.loss)

        else:
            output.loss.backward()

        optimizer.step()
        optimizer.zero_grad()
        step += 1

        ##### TODO: Apply linear learning rate decay #####
        scheduler.step()
        
        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print('Learning Rate :', optimizer.param_groups[0]['lr'])
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "/content/drive/MyDrive/bert/saved_model6" 
model.save_pretrained(model_save_dir)

Start Training ...


Learning Rate : 2e-05
Epoch 1 | Step 500 | loss = 1.164, acc = 0.583
Learning Rate : 2e-05
Epoch 1 | Step 1000 | loss = 0.726, acc = 0.722
Learning Rate : 2e-05
Epoch 1 | Step 1500 | loss = 0.629, acc = 0.746
Learning Rate : 2e-05
Epoch 1 | Step 2000 | loss = 0.582, acc = 0.745
Learning Rate : 2e-05
Epoch 1 | Step 2500 | loss = 0.581, acc = 0.772
Learning Rate : 2e-05
Epoch 1 | Step 3000 | loss = 0.594, acc = 0.765
Learning Rate : 1.2e-05
Epoch 1 | Step 3500 | loss = 0.496, acc = 0.795
Learning Rate : 1.2e-05
Epoch 1 | Step 4000 | loss = 0.488, acc = 0.779
Learning Rate : 1.2e-05
Epoch 1 | Step 4500 | loss = 0.531, acc = 0.779
Learning Rate : 1.2e-05
Epoch 1 | Step 5000 | loss = 0.469, acc = 0.792
Learning Rate : 1.2e-05
Epoch 1 | Step 5500 | loss = 0.504, acc = 0.792
Learning Rate : 1.2e-05
Epoch 1 | Step 6000 | loss = 0.511, acc = 0.788
Learning Rate : 7.2e-06
Epoch 1 | Step 6500 | loss = 0.500, acc = 0.781

Evaluating Dev Set ...



Validation | Epoch 1 | acc = 0.799


Learning Rate : 7.2e-06
Epoch 2 | Step 500 | loss = 0.217, acc = 0.886
Learning Rate : 7.2e-06
Epoch 2 | Step 1000 | loss = 0.232, acc = 0.880
Learning Rate : 7.2e-06
Epoch 2 | Step 1500 | loss = 0.178, acc = 0.903
Learning Rate : 7.2e-06
Epoch 2 | Step 2000 | loss = 0.214, acc = 0.880
Learning Rate : 4.319999999999999e-06
Epoch 2 | Step 2500 | loss = 0.200, acc = 0.888
Learning Rate : 4.319999999999999e-06
Epoch 2 | Step 3000 | loss = 0.193, acc = 0.897
Learning Rate : 4.319999999999999e-06
Epoch 2 | Step 3500 | loss = 0.201, acc = 0.895
Learning Rate : 4.319999999999999e-06
Epoch 2 | Step 4000 | loss = 0.193, acc = 0.892
Learning Rate : 4.319999999999999e-06
Epoch 2 | Step 4500 | loss = 0.212, acc = 0.890
Learning Rate : 4.319999999999999e-06
Epoch 2 | Step 5000 | loss = 0.225, acc = 0.884
Learning Rate : 2.5919999999999995e-06
Epoch 2 | Step 5500 | loss = 0.196, acc = 0.905
Learning Rate : 2.5919999999999995e-06
Epoch 2 | Step 6000 | loss = 0.208, acc = 0.891
Learning Rate : 2.59199


Validation | Epoch 2 | acc = 0.812
Saving Model ...


## Testing

In [9]:
model = torch.load('/content/drive/MyDrive/bert/saved_model3/pytorch_model.bin')
model1 = torch.load('/content/drive/MyDrive/bert/saved_model2/pytorch_model.bin')
model2 = torch.load('/content/drive/MyDrive/bert/saved_model1/pytorch_model.bin')
model3 = torch.load('/content/drive/MyDrive/bert/saved_model4/pytorch_model.bin')
model4 = torch.load('/content/drive/MyDrive/bert/saved_model6/pytorch_model.bin')
model5 = torch.load('/content/drive/MyDrive/bert/saved_model7/pytorch_model.bin')

In [10]:
import collections
model_en = collections.OrderedDict()
for key, value in model.items(): 
    if key in model1: 
     model_en[key] = (model[key] + model1[key] + model2[key] + model3[key])/4

In [11]:
# loading model
model = BertForQuestionAnswering.from_pretrained("hfl/chinese-roberta-wwm-ext-large").to(device)

model.load_state_dict(model_en)

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext-large were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl

<All keys matched successfully>

In [12]:
print("Evaluating Test Set ...")

result = []
out = []
model.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output))

result_file = "/content/result.csv"
#result_file = "/content/drive/MyDrive/result.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...



Completed! Result is in /content/result.csv


In [13]:
import pandas as pd
pd.set_option('display.max_rows', None)
#result = pd.read_csv('/content/drive/MyDrive/result_en.csv')
result = pd.read_csv("/content/result.csv")
unk = result[result['Answer'].str.contains('UNK') == True]
id = unk['ID'].to_list()
sub = unk['ID'].astype(str).tolist()
result['Answer'] = result['Answer'].apply(lambda x: str(x).replace('[UNK]', '.'))

In [14]:
import re
for i in sub:
    try:
        content = test_paragraphs[test_questions[int(i)]['paragraph_id']]
        pred = result.iloc[int(i)][1]
        print(pred)

        res = re.search(pred, content)
        a = res.span()
        start = a[0]
        end = a[-1]
        result.iloc[int(i),1] = test_paragraphs[test_questions[int(i)]['paragraph_id']][start:end]
    except:
      pass

溥.
目前沒有觀察到任何語言純.以力道來區分不同輔音
.人國
馬.
東晉常.
.稻
白.紀滅絕事件
抗佝.病
杭州.橋機場
蔡.
丁.
隋.帝
胡季.
英文縮寫首字母為「.·ㄎㄟ·ㄨㄞ」
梁.
.靼海峽
白.紀末滅絕事件
侏.紀
克里米亞.靼人
白.紀中期


In [15]:
result.iloc[int(250),1]

'溥儁'

In [16]:

result.to_csv('/content/result_postprocess.csv', index=False)

In [ ]:
# Ensemble


